# Attention temperature analysis

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
let plt = Python.import("matplotlib.pyplot")
let np = Python.import("numpy")

## Set training params

In [ ]:
let device = Device.defaultTFEager

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  50

In [ ]:
let datasetSize: DatasetSize = .full
let batchSize = 2

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

/// instantiate model
let modelSize = 128
let config = LangMotionTransformerConfig(
    vocabSize: vocabulary.count,
    nbJoints: 47, // TODO: get value from dataset
    nbMixtures: 20,
    layerCount: 6,
    modelSize: modelSize,
    feedForwardSize: 512,
    headCount: 4,
    dropoutProbability:  0.1,
    sentenceMaxPositionalLength: 100,
    motionMaxPositionalLength: 500,
    encoderSelfAttentionTemp: sqrt(Double(modelSize)),
    decoderSourceAttentionTemp: sqrt(Double(modelSize)),
    decoderSelfAttentionTemp: Double(modelSize)
)

let runName = "run_51"
let epoch = 150

let runURL = dataURL.appendingPathComponent("runs/Lang2motion/\(runName)", isDirectory: true)
let checkpointURL = runURL.appendingPathComponent("checkpoints", isDirectory: true)
let motionsURL = runURL.appendingPathComponent("generated_motions", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.e\(epoch)")

In [ ]:
/// load dataset
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    minMotionLength: 20,
    maxMotionLength: 50,
    trainTestSplit: 1.0,
    device: device
) { (motionSample: MotionSample) -> LangMotionBatch in    
    let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
    let (motionPart, target) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength, shiftMaskRight: true)
    let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
    let singleBatch = LangMotionBatch(source: source, target: target)
    return singleBatch
}

print("Dataset acquired.")

# Helpers

In [ ]:
// func tensorShow(_ tensor: Tensor<Float>, cmapRange: Float = 6.0) {
//     plt.figure(figsize: [5, 5])
//     plt.imshow(tensor.makeNumpyArray(), aspect: "auto", cmap: "Spectral", vmin: -cmapRange, vmax: cmapRange)
//     plt.show()
// }

func tensorShow(_ tensor: Tensor<Float>, cmapRange: Float = 6.0) {
    plt.figure(figsize: [5, 5])
    if cmapRange == 0.0 {
        plt.imshow(tensor.makeNumpyArray()) //, aspect: "auto") //, cmap: "Spectral")
    } else {
        plt.imshow(tensor.makeNumpyArray(), aspect: "auto", cmap: "Spectral", vmin: -cmapRange, vmax: cmapRange)
    }
    plt.show()
}

In [ ]:
extension LangMotionTransformer {
    public func getDecoderInput(sourceMask: Tensor<Float>, motionPart: LangMotionBatch.MotionPart, memory: Tensor<Float>) -> DecoderInput<Float> {
        var motionPartFeatures: Tensor<Float>

        // start flag, pos enc, current motion, padding with motion
        let shape = motionPart.motion.shape
        let (batchSize, numFrames) = (shape[0], shape[1])

        // motion positional encoding
        var motionPositionalEncodingVector = Tensor<Float>(repeating: 0.0, shape: [batchSize, numFrames, motionPositionalEncodingSize])
        motionPositionalEncodingVector = motionPositionalEncoding(motionPositionalEncodingVector)
        
        // compute padding
        let paddingSize = modelSize - (1 + motionPositionalEncodingSize + nbJoints)
        
        let multiplyBy = paddingSize/nbJoints + 1
        let motionFramePadding = motionPart.motion.tiled(multiples: [1, 1, multiplyBy])[0..., 0..., 0..<paddingSize]

        // stack everything together
        let tensorStack = [motionPart.startFlag, motionPositionalEncodingVector, motionPart.motion, motionFramePadding]
        let tmpMotionPartFeatures = Tensor<Float>(concatenating: tensorStack, alongAxis: 2)
        motionPartFeatures = tmpMotionPartFeatures

        motionPartFeatures = self.motionNorm(motionPartFeatures)
        
        let decoderInput = DecoderInput(sequence: motionPartFeatures, sourceMask: sourceMask, targetMask: motionPart.mask, memory: memory,
                                        sourceAttentionTemperature: Float(self.decoderSourceAttentionTemp), selfAttentionTemperature: Float(self.decoderSelfAttentionTemp))
        return decoderInput
    }
}

## Single batch

In [ ]:
let motionSample = dataset.motionSamples[0]
print("sampleID: \(motionSample.sampleID)")
print(motionSample.description)
print(motionSample.annotations)

In [ ]:
let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
let (motionPart, target) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength, shiftMaskRight: true)
let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
let singleBatch = LangMotionBatch(source: source, target: target)

In [ ]:
let trInput = singleBatch.source

In [ ]:
func tensorShow2(_ tensor: Tensor<Float>) {
    plt.imshow(tensor.makeNumpyArray(), cmap: "Spectral")
    plt.show()
}

# Encoder

In [ ]:
trInput.sentence.printSentence()

In [ ]:
let encoded = model.encode(input: trInput.sentence)
print(encoded.lastLayerOutput.shape)
tensorShow(encoded.lastLayerOutput[0], cmapRange: 0.0)

## self-attention mask

In [ ]:
trInput.sentence.mask

## encoder self-attention

In [ ]:
encoded.allLayerOutputs.count

In [ ]:
encoded.allLayerOutputs[0].attentionOutput!.result.shape

In [ ]:
encoded.allLayerOutputs[0].attentionOutput!.attentionScores.shape

In [ ]:
encoded.allLayerOutputs[0].attentionOutput!.attentionScores[0, 1].shape

In [ ]:
encoded.allLayerOutputs.map {tensorShow2($0.attentionOutput!.attentionProbs[0, 0])}

## encoder output

In [ ]:
encoded.allLayerOutputs.map {tensorShow($0.result[0], cmapRange: 0.0)}

In [ ]:
encoded.allLayerOutputs[5].result

# Decoder

In [ ]:
let decoded = model.decode(sourceMask: trInput.sourceAttentionMask, motionPart: trInput.motionPart, memory: encoded.lastLayerOutput)
print(decoded.allLayerOutputs.count)
decoded.lastLayerOutput.shape

## source sttention mask

In [ ]:
tensorShow2(trInput.sourceAttentionMask.squeezingShape(at: 0))

## self-attention decoder mask

In [ ]:
tensorShow2(trInput.motionPart.mask.squeezingShape(at: 0))

## decoder source attention

In [ ]:
decoded.allLayerOutputs.map {tensorShow2($0.sourceAttentionOutput!.attentionProbs[0, 0])}

## one source attention

In [ ]:
let oneSourceScores = decoded.allLayerOutputs[0].sourceAttentionOutput!.attentionScores[0, 0]
oneSourceScores.shape

In [ ]:
tensorShow2(oneSourceScores)

In [ ]:
let oneSourceScores2 = oneSourceScores.replacing(with: Tensor(onesLike: oneSourceScores) * 0.0, where: oneSourceScores .< Tensor<Float>([0.0]))

In [ ]:
tensorShow2(oneSourceScores2)

In [ ]:
oneSourceScores[10]

In [ ]:
softmax(oneSourceScores[10])*100

In [ ]:
plt.plot(softmax(oneSourceScores[10]).makeNumpyArray())
plt.show()

In [ ]:
oneSourceScores2[10]

In [ ]:
(oneSourceScores2[10] * 5).sum()

## decoder self-attention

In [ ]:
decoded.allLayerOutputs.map {tensorShow2($0.targetAttentionOutput!.attentionProbs[0, 0])}

## self-attention activations mins, maxs

In [ ]:
tensorShow2(trInput.motionPart.mask.squeezingShape(at: 0))

In [ ]:
let oneMaskProbs = decoded.allLayerOutputs[0].targetAttentionOutput!.attentionProbs[0, 0]
oneMaskProbs.shape

In [ ]:
tensorShow2(oneMaskProbs)

In [ ]:
plt.plot(oneMaskProbs[20].makeNumpyArray())
plt.show()

# oneMaskScores

In [ ]:
let oneMaskScores = decoded.allLayerOutputs[0].targetAttentionOutput!.attentionScores[0, 0]
oneMaskScores.shape

In [ ]:
tensorShow2(oneMaskScores)

In [ ]:
let oneMaskScores2 = oneMaskScores.replacing(with: Tensor(zerosLike: oneMaskScores), where: oneMaskScores .< Tensor<Float>([0.0]))

In [ ]:
tensorShow2(oneMaskScores2)

## loss

In [ ]:
let mixtureModelInput = Tensor<Float>(concatenating: decoded.allResults, alongAxis: 2)
let transformerOutput = LangMotionTransformerOutput(preds: model.mixtureModel(mixtureModelInput), encoded: encoded, decoded: decoded)

In [ ]:
// Loss function
let args = LossArgs(
        nb_joints: config.nbJoints,
        nb_mixtures: config.nbMixtures,
        mixture_regularizer_type: "None",  // ["cv", "l2", "None"]
        mixture_regularizer: 0.0,
        device: device
)

@differentiable(wrt: y_pred)
public func normalMixtureSurrogateLoss2(y_pred: MixtureModelPreds, y_true: LangMotionBatch.Target, args: LossArgs) -> (Tensor<Float>, Tensor<Float>) {
    // masking
    var y_pred = y_pred.squeezed()
    var y_true = y_true.squeezed()
    let ids = Tensor<Int32>(rangeFrom: 0, to: Int32(y_true.stops.shape[1]), stride: 1, on: args.device)
    let indices = ids.gathering(where: y_true.stops .!= Tensor(1, on: args.device))
    y_pred = y_pred.gathering(atIndices: indices, alongAxis: 1)
    y_true = y_true.gathering(atIndices: indices, alongAxis: 1)
    
    let loss = _normalMixtureSurrogateLoss(y_true: y_true, y_pred: y_pred, args: args)
    let mean_loss = loss.mean()
    return (mean_loss, loss)
}

In [ ]:
let (avg_loss, loss) = normalMixtureSurrogateLoss2(y_pred: transformerOutput.preds, y_true: singleBatch.target, args: args)

In [ ]:
print(avg_loss)
loss

In [ ]:
plt.plot(loss.scalars)
plt.show()